# Enrich events
Make sure to fill in the device_name (your Apple Watch device name) and that the following files are present:
- calendar.csv (an exported csv file from Outlook)
- export.xml (the exported XML file from Apple Health)

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

device_name = "Enter your wearable name here"

tree = ET.parse('export.xml')
root = tree.getroot()

hrv_data = []

for record in root.findall('Record'):
        
    if record.attrib.get('type') == 'HKQuantityTypeIdentifierHeartRateVariabilitySDNN':
        hrv_data.append({
            'start': record.attrib['startDate'],
            'end': record.attrib['endDate'],
            'value_ms': float(record.attrib['value']),
            'source': record.attrib.get('sourceName', 'unknown')
        })

hrv_df = pd.DataFrame(hrv_data)
hrv_df['start'] = pd.to_datetime(hrv_df['start'])
hrv_df['date'] = hrv_df['start'].dt.date

# store the first and last date for later use
first_date = hrv_df['date'].min()
last_date = hrv_df['date'].max()

# Bekijk de eerste rijen
hrv_df


In [ ]:
import pandas as pd

df = pd.read_csv('calendar.csv')

columns_to_drop = [
    'Reminder on/off', 'Reminder Date', 'Reminder Time', 'Meeting Resources',
    'Billing Information', 'Categories', 'Mileage', 'Priority', 'Private',
    'Sensitivity', 'Show time as'
]

df = df.drop(columns=columns_to_drop, errors='ignore')

# Merge Start Date and Start Time, End Date and End Time
df['Start'] = pd.to_datetime(df['Start Date'] + ' ' + df['Start Time'])
df['End'] = pd.to_datetime(df['End Date'] + ' ' + df['End Time'])

df = df.drop(columns=['Start Date', 'Start Time', 'End Date', 'End Time'], errors='ignore')

# Drop rows where 'All day event' is 'True'
df = df[df['All day event'] != True]

# Filter df to only keep rows between first_date and last_date
df = df[(df['Start'].dt.date >= first_date) & (df['End'].dt.date <= last_date)]

df.head(50)

In [ ]:

df['Start'] = df['Start'].dt.tz_localize('Europe/Amsterdam')  # Of jouw tijdzone
df['End'] = df['End'].dt.tz_localize('Europe/Amsterdam')

hrv_df['start'] = pd.to_datetime(hrv_df['start'])
hrv_df['end'] = pd.to_datetime(hrv_df['end'])

def get_hrv_stats(row):
    overlap = hrv_df[
        (hrv_df['end'] >= row['Start']) &
        (hrv_df['start'] <= row['End'])
    ]['value_ms']

    if len(overlap) == 0:
        return pd.Series({'HRV_mean': None, 'HRV_median': None})
    else:
        return pd.Series({
            'HRV_mean': overlap.mean(),
            'HRV_median': overlap.median()
        })

df[['HRV_mean', 'HRV_median']] = df.apply(get_hrv_stats, axis=1)

df_with_hrv = df[df['HRV_mean'].notna() | df['HRV_median'].notna()]

# Save unique Subject values to labelling.csv, for manual anonymization
pd.DataFrame({'Subject': df_with_hrv['Subject'].unique()}).to_csv('labelling.csv', index=False)

df_with_hrv

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Normalize Subject names for IPO events: case-insensitive grouping
ipo_df = df_with_hrv[df_with_hrv["Subject"].str.lower().str.startswith("ipo", na=False)].copy()
ipo_df["NormalizedSubject"] = ipo_df["Subject"].str.lower().str.strip().str.title()

# Define custom mappings for specific subjects
#custom_map = {
#    "ipo [online]": "Ipo David",
#    "ipo?": "Ipo David",
#    "ipo huckepuckebahnhof": "Ipo Julia",
#    "ipo zoo - inga annemarie lieneke": "Ipo Julia"
#}

# Apply custom mapping (case-insensitive)
#ipo_df["NormalizedSubject"] = ipo_df["NormalizedSubject"].replace(custom_map)

# Capitalize remaining categories nicely
ipo_df["NormalizedSubject"] = ipo_df["NormalizedSubject"].str.title()

plt.figure(figsize=(6,6))

# Group by initials (first letter after "Ipo")
ipo_df["Initial"] = ipo_df["NormalizedSubject"].apply(
    lambda x: x.split()[1][0] if len(x.split()) > 1 else x[0]
)

# Ensure groups are sorted by Start before plotting lines
for initial, group in ipo_df.groupby("Initial"):
    group_sorted = group.sort_values("Start")
    plt.plot(group_sorted["Start"], group_sorted["HRV_median"],
             linestyle="--", marker=None, alpha=0.4, color="gray")
    for i, row in group_sorted.iterrows():
        plt.text(row["Start"], row["HRV_median"], initial, fontsize=12, fontweight="bold",
                 ha="center", va="center", color="black")

# Set axis limits properly
plt.xlim(ipo_df["Start"].min() - pd.Timedelta(days=1), ipo_df["Start"].max() + pd.Timedelta(days=1))
plt.ylim(ipo_df["HRV_median"].min() - 5, ipo_df["HRV_median"].max() + 5)

plt.title("HRV Values Across Single Meeting Type", fontsize=14, fontweight="bold")
plt.xlabel("Date")
plt.ylabel("Median HRV (ms)")
plt.xticks(rotation=45)
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()



In [ ]:
# Normalize Subject for robust matching (lowercase, strip spaces)
df_with_hrv["NormalizedSubject"] = df_with_hrv["Subject"].str.lower().str.strip()

# Define meeting patterns to match (lowercase for comparison)
meeting_subjects = ["mt", "office update (via teams)", "update front office"]

# Filter rows where Subject matches one of the meeting types (case-insensitive)
recurring_df = df_with_hrv[df_with_hrv["NormalizedSubject"].isin(meeting_subjects)].copy()

# Create short labels
label_map = {
    "mt": "M",
    "office update (via teams)": "O",
    "update front office": "F"
}
recurring_df["Label"] = recurring_df["NormalizedSubject"].map(label_map)

# Only proceed if we found matches
if not recurring_df.empty:
    plt.figure(figsize=(6,6))

    # Plot chronological connections
    for label, group in recurring_df.groupby("Label"):
        group_sorted = group.sort_values("Start")
        plt.plot(group_sorted["Start"], group_sorted["HRV_median"],
                 linestyle="--", marker=None, alpha=0.4, color="gray")
        for i, row in group_sorted.iterrows():
            plt.text(row["Start"], row["HRV_median"], row["Label"],
                     fontsize=12, fontweight="bold", ha="center", va="center")

    # Set axis limits properly
    plt.xlim(recurring_df["Start"].min() - pd.Timedelta(days=1), recurring_df["Start"].max() + pd.Timedelta(days=1))
    plt.ylim(recurring_df["HRV_median"].min() - 5, recurring_df["HRV_median"].max() + 5)

    plt.title("HRV Values Across Recurrent Meetings", fontsize=14, fontweight="bold")
    plt.xlabel("Date")
    plt.ylabel("Median HRV (ms)")
    plt.xticks(rotation=45)
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.tight_layout()
    plt.show()
else:
    print("No matching recurrent meetings found. Please check Subject names.")


# Enrich cases

In [ ]:
resting_hr_data = []

for record in root.findall('Record'):
    if (record.attrib.get('type') == 'HKQuantityTypeIdentifierRestingHeartRate' and
        record.attrib.get('sourceName') == device_name):
        
        resting_hr_data.append({
            'start': pd.to_datetime(record.attrib['startDate']),
            'value_bpm': float(record.attrib['value']),
            'source': record.attrib.get('sourceName', 'unknown')
        })

# Zet in DataFrame
restingHR_df = pd.DataFrame(resting_hr_data)

# Voeg datumkolom toe
restingHR_df['date'] = restingHR_df['start'].dt.date

# Bereken gemiddelde rusthartslag per dag (je kunt ook .min() of .first() doen)
restingHR_df = restingHR_df.groupby('date', as_index=False)['value_bpm'].mean()

# Optioneel: hernoem kolommen voor consistentie
restingHR_df = restingHR_df.rename(columns={'value_bpm': 'resting_hr_bpm'})

# Sort descending by resting_hr_bpm
restingHR_df = restingHR_df.sort_values('resting_hr_bpm', ascending=False).reset_index(drop=True)

# Bekijk het resultaat
restingHR_df

In [ ]:
sleep_data = []

for record in root.findall('Record'):
    if (record.attrib.get('type') == 'HKCategoryTypeIdentifierSleepAnalysis' and
        record.attrib.get('sourceName') == device_name):
        sleep_data.append({
            'type': 'sleep',
            'start': record.attrib['startDate'],
            'end': record.attrib['endDate'],
            'value': record.attrib['value'],
            'source': record.attrib.get('sourceName', 'unknown')
        })

if sleep_data:
    sleep_df = pd.DataFrame(sleep_data)
    sleep_df['start'] = pd.to_datetime(sleep_df['start'])
    sleep_df['end'] = pd.to_datetime(sleep_df['end'])
    display(sleep_df)
else:
    print("No sleep records found for device.")

In [ ]:
import pandas as pd

sleep_data = []

for record in root.findall('Record'):
    if (record.attrib.get('type') == 'HKCategoryTypeIdentifierSleepAnalysis' and
        record.attrib.get('sourceName') == device_name):
        sleep_data.append({
            'type': 'sleep',
            'start': record.attrib['startDate'],
            'end': record.attrib['endDate'],
            'value': record.attrib['value'],
            'source': record.attrib.get('sourceName', 'unknown')
        })

if sleep_data:
    sleep_df = pd.DataFrame(sleep_data)
    sleep_df['start'] = pd.to_datetime(sleep_df['start'])
    sleep_df['end'] = pd.to_datetime(sleep_df['end'])

    # Compute duration in minutes
    sleep_df['duration_min'] = (sleep_df['end'] - sleep_df['start']).dt.total_seconds() / 60

    # Assign each record to the "night of" its start date
    # If someone goes to bed after midnight, shift back to the previous date
    sleep_df['night'] = sleep_df['start'].dt.date
    sleep_df.loc[sleep_df['start'].dt.hour < 12, 'night'] = sleep_df.loc[sleep_df['start'].dt.hour < 12, 'night'] - pd.to_timedelta(1, unit='d')

    # Aggregate per night and sleep stage (value)
    sleep_agg = sleep_df.groupby(['night', 'value'])['duration_min'].sum().reset_index()

    # Pivot to have sleep stages as columns
    sleep_pivot = sleep_agg.pivot(index='night', columns='value', values='duration_min').fillna(0)

else:
    print("No sleep records found for device.")


In [ ]:
sleep_agg

In [ ]:
# Ensure datetime
df['Start'] = pd.to_datetime(df['Start'])
df['End'] = pd.to_datetime(df['End'])

# Keep only events that start and end on the same calendar date
df = df[df['Start'].dt.date == df['End'].dt.date].copy()

# 1. Build daily event sequences from df
df['workday'] = df['Start'].dt.date
day_sequences = (
    df.sort_values('Start')
      .groupby('workday')['Subject']
      .apply(list)
      .reset_index()
      .rename(columns={'Subject': 'event_sequence'})
)

# 2. Reset index for sleep_pivot and ensure 'night' is date
sleep_pivot = sleep_pivot.reset_index()
sleep_pivot['night'] = pd.to_datetime(sleep_pivot['night']).dt.date

# 3. Shift sleep nights back by one day so each workday is linked to the following night
sleep_pivot['workday'] = sleep_pivot['night'] - pd.to_timedelta(1, unit='d')

# Add a TotalSleep column: sum Core, Deep, and REM
sleep_pivot['TotalSleep'] = (
    sleep_pivot.get('HKCategoryValueSleepAnalysisAsleepCore', 0) +
    sleep_pivot.get('HKCategoryValueSleepAnalysisAsleepDeep', 0) +
    sleep_pivot.get('HKCategoryValueSleepAnalysisAsleepREM', 0)
)


# 4. Merge workday sequences with the *following* night's sleep
merged = pd.merge(
    day_sequences,
    sleep_pivot,
    on='workday',
    how='left'
)

# Drop duplicate 'night' column if desired
merged = merged.drop(columns=['night'])

merged


# Enrich activities

In [ ]:
workout_data = []

for workout in root.findall('Workout'):
    if workout.attrib.get('sourceName') == device_name:
        # sla op
        workout_data.append({
            'type': 'workout',
            'start': workout.attrib['startDate'],
            'end': workout.attrib['endDate'],
            'activity': workout.attrib.get('workoutActivityType'),
            'duration_min': float(workout.attrib.get('duration', 0)),
            'source': workout.attrib.get('sourceName', 'unknown')
        })

workout_df = pd.DataFrame(workout_data)
workout_df['start'] = pd.to_datetime(workout_df['start'])
workout_df['end'] = pd.to_datetime(workout_df['end'])

workout_df

# Build event log

In [ ]:
# Load labelling.csv
label_df = pd.read_csv('labelled.csv', sep=';')

# Create mapping from Subject to Label
subject_to_label = dict(zip(label_df['Subject'], label_df['Label']))

# Replace Subject with Label in df_with_hrv
df_with_hrv['Subject'] = df_with_hrv['Subject'].map(subject_to_label)

# Copy df_with_hrv and drop specified columns
event_log = df_with_hrv.drop(
    columns=[
        'NormalizedSubject',
        'All day event',
        'Meeting Organizer',
        'Required Attendees',
        'Optional Attendees',
        'Description',
        'Location',
        'HRV_mean'
    ],
    errors='ignore'
)

# Rename NormalizedSubject to Activity in event_log
event_log = event_log.rename(columns={'Subject': 'Activity'})

# Prepare workout_df for appending
workout_events = workout_df[['start', 'end', 'activity']].copy()
workout_events = workout_events.rename(columns={
    'start': 'Start',
    'end': 'End',
    'activity': 'Activity',
})

# Add missing columns to workout_events to match event_log structure
for col in event_log.columns:
    if col not in workout_events.columns:
        workout_events[col] = None
workout_events = workout_events[event_log.columns]  # Ensure column order

# Append workout_events to event_log
event_log = pd.concat([event_log, workout_events], ignore_index=True)

# Clean up Activity values: remove 'HKWorkoutActivityType' prefix
event_log['Activity'] = event_log['Activity'].replace(
    r'^HKWorkoutActivityType', '', regex=True
).str.strip()

event_log

In [ ]:
def remove_tz(dt):
    try:
        dt = pd.to_datetime(dt)
        if pd.isnull(dt):
            return pd.NaT
        if hasattr(dt, 'tzinfo') and dt.tzinfo is not None:
            return dt.tz_localize(None)
        return dt
    except Exception:
        return pd.NaT

event_log['Start'] = event_log['Start'].apply(remove_tz)
event_log['End'] = event_log['End'].apply(remove_tz)
event_log['Workday'] = event_log['Start'].dt.strftime('%Y-%m-%d')

# Filter out Workdays that contain only Activities starting with 'Private' or 'Sports'
def is_only_private_or_sports(activities):
    return all(a.startswith('Private') or a.startswith('Sports') for a in activities)

def has_work_activity(activities):
    return any(a.startswith('Work') for a in activities)

# Group by Workday and filter
filtered_days = (
    event_log.groupby('Workday')['Activity']
    .apply(list)
    .reset_index()
)

# Keep only Workdays with at least one 'Work' activity
valid_workdays = filtered_days[filtered_days['Activity'].apply(has_work_activity)]['Workday']

# Filter event_log
event_log = event_log[event_log['Workday'].isin(valid_workdays)]

# Prepare sleep_df for appending to event_log
sleep_events = sleep_df[['start', 'end', 'value', 'night']].copy()
sleep_events = sleep_events.rename(columns={
    'start': 'Start',
    'end': 'End',
    'value': 'Activity',
    'night': 'Workday'
})

# Add missing columns to sleep_events to match event_log structure
for col in event_log.columns:
    if col not in sleep_events.columns:
        sleep_events[col] = None
sleep_events = sleep_events[event_log.columns]  # Ensure column order

# Append sleep_events to event_log
event_log = pd.concat([event_log, sleep_events], ignore_index=True)

event_log['Start'] = event_log['Start'].apply(remove_tz)
event_log['End'] = event_log['End'].apply(remove_tz)

event_log

In [ ]:
event_log['RestingHR'] = None

# Create a mapping from date to resting_hr_bpm
resting_hr_map = dict(zip(restingHR_df['date'].astype(str), restingHR_df['resting_hr_bpm']))

# Assign RestingHR to event_log based on Workday
event_log['RestingHR'] = event_log['Workday'].map(resting_hr_map)

event_log

In [ ]:
# Create mappings from merged for each sleep variable
total_sleep_map = dict(zip(merged['workday'].astype(str), merged['TotalSleep']))
awake_map = dict(zip(merged['workday'].astype(str), merged.get('HKCategoryValueSleepAnalysisAwake', pd.Series())))
deep_sleep_map = dict(zip(merged['workday'].astype(str), merged.get('HKCategoryValueSleepAnalysisAsleepDeep', pd.Series())))

# Assign to event_log based on Workday
event_log['TotalSleep'] = event_log['Workday'].map(total_sleep_map)
event_log['Awake'] = event_log['Workday'].map(awake_map)
event_log['DeepSleep'] = event_log['Workday'].map(deep_sleep_map)

# Clean up Activity values: remove 'HKCategoryValueSleepAnalysisAsleepCore' prefix
event_log['Activity'] = event_log['Activity'].replace(
    r'^HKCategoryValueSleepAnalysis', '', regex=True
).str.strip()

event_log

In [ ]:
event_log.to_csv('event_log.csv', index=False)

event_log

In [ ]:
# Exclude sleep-related activities from event_log
exclude_activities = ['AsleepREM', 'Awake', 'AsleepCore', 'AsleepDeep', 'AsleepUnspecified']

filtered_event_log = event_log[~event_log['Activity'].isin(exclude_activities)]

filtered_event_log

In [ ]:
import matplotlib.pyplot as plt

# Select the workdays of interest
selected_days = ['2024-11-20', '2025-05-22', '2024-12-11', '2024-12-31', '2025-06-13']
selected_df = event_log[event_log['Workday'].isin(selected_days)]

# Aggregate by Workday (in case there are multiple activities per day)
agg = selected_df.groupby('Workday')[['RestingHR', 'TotalSleep', 'Awake', 'DeepSleep']].first().reset_index()

# Calculate overall averages for reference
overall_means = event_log[['RestingHR', 'TotalSleep', 'Awake', 'DeepSleep']].mean()

# Plot: metrics underneath each other (vertical subplots)
fig, axes = plt.subplots(4, 1, figsize=(2, 6), sharex=True)
metrics = ['RestingHR', 'TotalSleep', 'Awake', 'DeepSleep']

for ax, metric in zip(axes, metrics):
    ax.bar(agg['Workday'], agg[metric], label='Selected days')
    ax.axhline(overall_means[metric], color='red', linestyle='--', label='Overall mean')
    #ax.set_title(metric)
    ax.set_ylabel(metric)
    ax.grid(True, linestyle='--', alpha=0.5)
    #ax.legend()

axes[-1].set_xticklabels(agg['Workday'], rotation=90)

In [ ]:
# Only include activities that occur at least twice
activity_counts = filtered_event_log['Activity'].value_counts()
valid_activities = activity_counts[activity_counts >= 2].index

filtered_event_log_twice = filtered_event_log[filtered_event_log['Activity'].isin(valid_activities)]

# Calculate average and median RestingHR, TotalSleep, Awake, and DeepSleep for each unique Activity
summary = (
    filtered_event_log_twice
    .groupby('Activity')[['RestingHR', 'TotalSleep', 'Awake', 'DeepSleep']]
    .agg(['mean', 'median'])
    .reset_index()
)

summary.to_csv('activity_summary.csv', index=False)
summary